In [86]:
import pandas as pd
!pip install apyori
from apyori import apriori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


каждая транзакция представляет собой несколько строк, которые надо объединять по номеру транзакции в одну строку

In [94]:
df = pd.read_excel('transactions.xlsx')
df.head(3)

,Номер заказа,Категория,Товар,Цена
0,SO61269,Шлемы,Sport-100,53.99
1,SO61269,Трикотажные изделия,Куртка с длинными руковами,49.99
2,SO61270,Крылья,Набор крыльев для горного велосипеда,21.98


In [195]:
transactions = df.groupby(['Номер заказа'])['Товар'].apply(list).values.tolist()
transactions[:4]


[['Sport-100', 'Куртка с длинными руковами'],
 ['Набор крыльев для горного велосипеда'],
 ['Шина LL Road', 'Ремонтный комплект'],
 ['Шина для горного велосипеда', 'Ремонтный комплект']]

In [241]:
# rules = list(apriori(transactions, min_support=0.001,  min_confidence=0.75, min_lift=1.07, min_length=2))
rules = list(apriori(transactions, min_support=0.02,  min_confidence=0.02, min_lift=1.001, max_length = 2, min_length = 2))
# rules = list(apriori(transactions, min_support=0.01,  min_confidence=0.01, min_lift=0))
# rules = list(apriori(transactions, min_support = 0.03, min_confidence= 0.03, min_lift = 1.5, min_length = 2, max_length = 2))
# rules = list(apriori(transactions, min_support = 0.03, min_confidence= 0.01, min_lift = 0))

In [242]:
# оформим вывод более наглядно
for item in rules:
    pair = item[2][0]
    base = [x for x in pair.items_base]
    add = [x for x in pair.items_add]
    base_str = ', '.join(base)
    add_str = ', '.join(add)
    print('({}) ==> ({}) sup = {}, conf = {}, lift = {}'.format(base_str, add_str, str(round(item[1],3)), str(round(item[2][0][2],3)), str(round(item[2][0][3],3))))



(HL шина для горного велосипеда) ==> (Mountain-200) sup = 0.024, conf = 0.385, lift = 3.409
(HL шина для горного велосипеда) ==> (Sport-100) sup = 0.02, conf = 0.324, lift = 1.113
(HL шина для горного велосипеда) ==> (Шина для горного велосипеда) sup = 0.042, conf = 0.676, lift = 4.954
(HL шина для дорожн. велосипеда) ==> (Шина для дорожн. велосипеда) sup = 0.025, conf = 0.704, lift = 6.673
(LL шина для горного велосипеда) ==> (Шина для горного велосипеда) sup = 0.021, conf = 0.555, lift = 4.065
(ML шина для горного велос.) ==> (Шина для горного велосипеда) sup = 0.033, conf = 0.658, lift = 4.819
(ML шина для дорожн. велосипеда) ==> (Шина для дорожн. велосипеда) sup = 0.028, conf = 0.681, lift = 6.454
(Mountain-200) ==> (Бутылка для воды) sup = 0.026, conf = 0.234, lift = 1.218
(Mountain-200) ==> (Крепление для бутыкли (горн. вел.)) sup = 0.033, conf = 0.292, lift = 3.172
(Mountain-200) ==> (Набор крыльев для горного велосипеда) sup = 0.034, conf = 0.297, lift = 3.134
(Road-550-W) ==> 

In [243]:
results = rules
def inspect(results):
    lhs         =[tuple(result[2][0][0])[0] for result in results]
    rhs         =[tuple(result[2][0][1])[0] for result in results]
    supports    =[result[1] for result in results]
    confidences =[result[2][0][2] for result in results]
    lifts        =[result[2][0][3] for result in results]
    return list (zip(lhs, rhs, supports, confidences, lifts))
    

resultsinDataFrame = pd.DataFrame(inspect(results), columns = ["Left hand side", "Right hand side", "Support", "Confidence", "Lift"]).round(3)
# resultsinDataFrame = pd.DataFrame(inspect(results), columns = ["Support", "Confidence", "Lift"]).round(3)
# resultsinDataFrame

In [244]:
resultsinDataFrame
# resultsinDataFrame.nlargest(n = 30, columns = "Lift")
# resultsinDataFrame.nlargest(n = 30, columns = "Support") # Высокий Support - обеспечивает высокую достоверность правила x -> y. при высоком Lift  - это 
# resultsinDataFrame.nlargest(n = 24, columns = "Support")


,Left hand side,Right hand side,Support,Confidence,Lift
0,HL шина для горного велосипеда,Mountain-200,0.024,0.385,3.409
1,HL шина для горного велосипеда,Sport-100,0.020,0.324,1.113
2,HL шина для горного велосипеда,Шина для горного велосипеда,0.042,0.676,4.954
3,HL шина для дорожн. велосипеда,Шина для дорожн. велосипеда,0.025,0.704,6.673
4,LL шина для горного велосипеда,Шина для горного велосипеда,0.021,0.555,4.065
5,ML шина для горного велос.,Шина для горного велосипеда,0.033,0.658,4.819
6,ML шина для дорожн. велосипеда,Шина для дорожн. велосипеда,0.028,0.681,6.454
7,Mountain-200,Бутылка для воды,0.026,0.234,1.218
8,Mountain-200,Крепление для бутыкли (горн. вел.),0.033,0.292,3.172
9,Mountain-200,Набор крыльев для горного велосипеда,0.034,0.297,3.134


[RelationRecord(items=frozenset({'HL шина для горного велосипеда', 'Mountain-200'}), support=0.024061302681992337, ordered_statistics=[OrderedStatistic(items_base=frozenset({'HL шина для горного велосипеда'}), items_add=frozenset({'Mountain-200'}), confidence=0.38480392156862747, lift=3.409158979273991), OrderedStatistic(items_base=frozenset({'Mountain-200'}), items_add=frozenset({'HL шина для горного велосипеда'}), confidence=0.21317040054310932, lift=3.409158979273991)]),
 RelationRecord(items=frozenset({'Sport-100', 'HL шина для горного велосипеда'}), support=0.020229885057471263, ordered_statistics=[OrderedStatistic(items_base=frozenset({'HL шина для горного велосипеда'}), items_add=frozenset({'Sport-100'}), confidence=0.3235294117647059, lift=1.112825203882291), OrderedStatistic(items_base=frozenset({'Sport-100'}), items_add=frozenset({'HL шина для горного велосипеда'}), confidence=0.06958355297838692, lift=1.1128252038822908)]),
 RelationRecord(items=frozenset({'Шина для горного 